In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import transformers
import torch
import torch.nn.functional as F
import math

In [40]:
math.comb(25,4)

12650

In [2]:
cipher = """Xoi Doijtooi Tooijto-Ntoit-oo Toooj bot iott boi Giüobooj boi Öotoiooboooi to Jobi 4014 toit 40.000 Nojoioojojoo oot bto Xäoboi noo Ditnottooboo to Xootiobjoob joiobioopt. Dti zoo Jobioioobo iojjoo oi ioob 100.000 Dojojoo ioto. 4044 taootoo ooob otoooj io ntojo Nojoibäoboi btozotooooo. Xoiüpoi btoooi bot Toooj poiotti Iooioobo Tooijtoiootoboi, Loboitottoooo tüi Tjottioootoi oob Uäioooooooo pot iotooo Kooboo toitojjtoit. Noboo to oäobitoo Jobi taooto oto Iotj btoioi Dojojoo oji otooi boi jiaßtoo ntitoojjoo Kiottnoito to Toiooo oo Tooijtoooitt oottiotoo.
"""
text = """Das Berliner Energie-Start-up Enpal hat seit der Gründung des Unternehmens im Jahr 2017 fast 70.000 Solaranlagen auf die Dächer von Privatkunden in Deutschland geschraubt. Bis zum Jahresende sollen es rund 100.000 Anlagen sein. 2025 könnten noch einmal so viele Solardächer hinzukommen. Darüber hinaus hat Enpal bereits Tausende Energiespeicher, Ladestationen für Elektroautos und Wärmepumpen bei seinen Kunden installiert. Schon im nächsten Jahr könnte ein Teil dieser Anlagen als eines der größten virtuellen Kraftwerke in Europa am Energiemarkt auftreten.
"""

In [172]:
cipher_list = cipher.lower().split(' ')
cipher_list = [word.strip('.').strip('\n') for word in cipher_list ] 

text_list = text.lower().split(' ')
text_list = [word.strip('.').strip('\n') for word in text_list ] 

pairs = {(i, k) for i,k in zip(cipher_list, text_list)}
rules = {}

for pair in pairs:
    for c, s in zip(pair[0],pair[1]):
        if c in rules:
            if s not in rules[c]:
                rules[c] += s
        else:
            rules[c] = s
rules['r'] = 'r'

In [173]:
abc = ''.join([chr(ord('a')+i) for i in range(ord('z') - ord('a')+1)])
for i in range(ord('z') - ord('a')):
    ch = chr(ord('a')+i)
    if ch not in rules:
        print(ch)
        rules[ch] = abc

c
e
f
h
m
q
s
v
w
y


In [174]:
rules

{'o': 'enmaupoc',
 't': 'ietfk',
 'j': 'lgj',
 'd': 'bpa',
 'i': 'rst',
 '-': '-',
 'n': 'svw',
 '4': '275',
 '0': '0',
 '1': '1',
 'b': 'hd',
 'ü': 'ü',
 'z': 'z',
 'p': 'b',
 '.': '.',
 'ä': 'ä',
 'a': 'ö',
 'ß': 'ß',
 'x': 'd',
 'u': 'w',
 ',': ',',
 'g': 'g',
 'l': 'l',
 'k': 'k',
 'ö': 'u',
 'r': 'r',
 'c': 'abcdefghijklmnopqrstuvwxyz',
 'e': 'abcdefghijklmnopqrstuvwxyz',
 'f': 'abcdefghijklmnopqrstuvwxyz',
 'h': 'abcdefghijklmnopqrstuvwxyz',
 'm': 'abcdefghijklmnopqrstuvwxyz',
 'q': 'abcdefghijklmnopqrstuvwxyz',
 's': 'abcdefghijklmnopqrstuvwxyz',
 'v': 'abcdefghijklmnopqrstuvwxyz',
 'w': 'abcdefghijklmnopqrstuvwxyz',
 'y': 'abcdefghijklmnopqrstuvwxyz'}

In [180]:
raw_dictionary = None
path_to_dict = '../aime-analysis/wortliste.txt'
with open(path_to_dict, 'r') as f:
    raw_dictionary = f.readlines()

raw_dictionary = [word.strip() for word in raw_dictionary]

In [208]:
def find_dictionary_word(cipher, raw_dictionary=raw_dictionary, rules=rules):
    lower_cipher = cipher.lower().strip()
    restricted_dict = [word.lower() for word in raw_dictionary if len(lower_cipher) <= len(word) <= len(lower_cipher)+1]
    for i, letter in enumerate(lower_cipher):
        tmp_dict = []
        allowed = rules[letter]
        for word in restricted_dict:
            if word[i] in allowed and word not in tmp_dict:
                tmp_dict += [word] 
        restricted_dict = tmp_dict
    return restricted_dict

find_dictionary_word('oäobitoo')

['nächstem', 'nächsten', 'nächstens']

In [177]:
def is_valid_german_word(word, abbr = False):
    """
    Validates if a word follows German letter combination rules.
    Returns True if the word is valid, False if it contains impossible combinations.
    """
    vowels = 'aeiouäöüy'
    consonants = 'bcdfghjklmnpqrstvwxz'
    word = word.lower()
    
    # 1. "dt" at the beginning of a word
    if word.startswith('dt'):
        return False
    
    # 2. "ck" at the beginning of a word
    if word.startswith('ck'):
        return False
    
    # 3. "tz" at the end without a vowel before it
    if word.endswith('tz') and len(word) > 2:
        if word[-3] not in vowels:
            return False
    
    # 4. "pf", "sch", "st", "sp" at the end without a vowel before them
    end_groups = ['pf', 'st', 'sp']
    if any(word.endswith(group) and len(word) > len(group) and word[-(len(group)+1)] not in vowels for group in end_groups):
        return False
    if word.endswith('sch') and len(word) > 3 and word[-4] not in vowels:
        return False
    
    # 5. "ä", "ö", "ü" followed immediately by another vowel
    for i in range(len(word) - 1):
        if word[i] in 'äöü' and word[i+1] in vowels:
            return False
    
    # 7. "q" without a following "u"
    for i in range(len(word) - 1):
        if word[i] == 'q' and word[i+1] != 'u':
            return False
    if word.endswith('q'):
        return False

    if len(word) > 3 and all([c in vowels for c in word]) and not abbr:
        return False

    if all([c in consonants for c in word]) and not abbr:
        return False
    
    # 12. Triple consonants without a hyphen
    for i in range(len(word) - 2):
        if (word[i] in consonants and 
            word[i+1] in consonants and 
            word[i+2] in consonants and
            word[i] == word[i+1] == word[i+2]):
            return False
    
    return True

def produce_variants(word, rules=rules):
    result = []
    vectors = [list(rules[c]) for c in word]
    for prod in it.product(*vectors):
        word = ''.join(prod)
        if is_valid_german_word(word):
            yield word


len([var for var in produce_variants('ronojo')])

7289856

In [7]:
model_name = "dbmdz/german-gpt2"
model = transformers.GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [221]:
def check_model_variants(phrase, word, predictions, model=model, tokenizer=tokenizer, n_tokens = 5, cut=0.90):
    next_token_logits = predictions[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    top_k_tokens = torch.topk(next_token_probs, k=n_tokens)  # Get top 10 possible next tokens
    # Decode the token IDs to words
    next_words = [tokenizer.decode([token_id], skip_special_tokens=True) for token_id in top_k_tokens.indices]
    
    # Print the next possible words and their probabilities
    for var_word, var_prob in zip(next_words, top_k_tokens.values):
        var_word = var_word.strip()
        if len(var_word) == len(word) and var_prob.item() > cut:
            print(f"Word: {var_word} Probability: {var_prob.item()}")
            return  var_word, var_prob
    return None, -1

def decipher(text, model=model, tokenizer=tokenizer, rules=rules, prior='\n'):
    result = prior

    raw_text_list = text.split(' ')
    text_list = []
    punctuations = [',', '.', ';', '\n', '?', '!', '"']
    for word in raw_text_list:
        if word[-1] in punctuations:
            for punct in punctuations:
                if word.endswith(punct):
                    text_list += [word.strip(punct), punct, ' ']
        elif '-' in word:
            text_list += word.split('-')  
        else:
            text_list += [word]    
    
    for word in text_list:
        if not word or word == ' ':
            continue
        if word in punctuations:
            result += word
            continue

        inputs = tokenizer(result, return_tensors='pt')
        # Get model outputs (logits)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        
        # Convert logits to probabilities using softmax
        probabilities = F.softmax(logits, dim=-1)
        variants = find_dictionary_word(word.lower())
        print(f'{len(variants)} dictionary variants')
        if len(variants) == 0:
            variants = produce_variants(word.lower())
            
        best_var, max_prob = check_model_variants(result, word, logits)
        #best_var = None
        if best_var is None:
            max_prob = 0.0
            for variant in variants:
                tmp = ' '
                if word.istitle():
                    tmp += variant.capitalize()
                else:
                    tmp += variant
                #next_token_id = tokenizer.encode(tmp)[0]  # Token ID for the next token "example"
                #probability = probabilities[0, -1, next_token_id].item()
                next_token_ids = tokenizer.encode(tmp)
                probability = 1.0
                for token_id in next_token_ids:
                    # Assuming we are calculating the probability of the next tokens after the input text
                    probability *= probabilities[0, -1, token_id].item()
    
                if probability > max_prob:
                    max_prob = probability
                    best_var = tmp
        print(result.strip()[len(prior)-1:], '|', best_var, max_prob)
        if result[-1] != ' ' and best_var[0] != ' ':
            result += ' '
        result += best_var
    return result

decipher('Uoiboiio')

0 dictionary variants
 |  Werdersa 2.315834125024193e-11


'\n Werdersa'

In [ ]:
decipher('Noboo to oäobitoo Jobi taooto oto Iotj btoioi Dojojoo oji otooi boi jiaßtoo ntitoojjoo Kiottnoito to Toiooo oo Tooijtoooitt oottiotoo', 
         prior='Aus hunderttausend Solardächern wird ein Großkraftwerk.\nDas Berliner Energieunternehmen Enpal und die Münchner Softwarefirma Entrix gründen ein Joint Venture / Künstliche Intelligenz soll virtuelles Kraftwerk optimieren.\n'
        )

In [223]:
decipher('Xoob noi ooiitoit, nooo bto Küoitjtobo Iotojjtjooz ott tbioo Uoiboiiojoo tüi boi ntitoojjo Kiottnoit boob otoooj booopoojtojt oob boi Djjoittbooi oot boo Tooijtoooitt Ntioo noitoott, boo boi Koobo noo Toooj otjootjtob tüi bto Uoiobooiobtoo pooattjt?',
        prior='Das Berliner Energieunternehmen Enpal und die Münchner Softwarefirma Entrix gründen ein Joint Venture / Künstliche Intelligenz soll virtuelles Kraftwerk optimieren.\n')


7 dictionary variants
 |  Doch 7.34875413854752e-08
74 dictionary variants

 Doch |  was 0.0174531489610672
31 dictionary variants

 Doch was |  passiert 0.0320647694170475
85 dictionary variants
Word: wenn Probability: 0.9662197828292847

 Doch was passiert, | wenn tensor(0.9662)
64 dictionary variants

 Doch was passiert, wenn |  die 0.15111319720745087
5 dictionary variants

 Doch was passiert, wenn die |  Künstliche 3.7553607311358754e-12
0 dictionary variants
Word: Intelligenz Probability: 0.9925306439399719

 Doch was passiert, wenn die Künstliche | Intelligenz tensor(0.9925)
76 dictionary variants

 Doch was passiert, wenn die Künstliche Intelligenz |  mit 0.010966898873448372
8 dictionary variants

 Doch was passiert, wenn die Künstliche Intelligenz mit |  ihrem 0.04068536311388016
0 dictionary variants


KeyboardInterrupt: 

In [ ]:

coded_text = """Xoi Doijtooi Tooijto-Ntoit-oo Toooj bot iott boi Giüobooj boi Öotoiooboooi to Jobi 4014 toit 40.000 Nojoioojojoo oot bto Xäoboi noo Ditnottooboo to Xootiobjoob joiobioopt. Dti zoo Jobioioobo iojjoo oi ioob 100.000 Dojojoo ioto. 4044 taootoo ooob otoooj io ntojo Nojoibäoboi btozotooooo. Xoiüpoi btoooi bot Toooj poiotti Iooioobo Tooijtoiootoboi, Loboitottoooo tüi Tjottioootoi oob Uäioooooooo pot iotooo Kooboo toitojjtoit. Noboo to oäobitoo Jobi taooto oto Iotj btoioi Dojojoo oji otooi boi jiaßtoo ntitoojjoo Kiottnoito to Toiooo oo Tooijtoooitt oottiotoo.
"Uooo ooo Ttozojbooibojto jojoo bto Ntioooiotitoino oottotoit, tooo ooo jooz ntojo Xtojo joi otobt oooboo, notj boi Ntiooooitt otobt tüi iojobo Giaßoo joboobt tit", iojt Doojooto Voijo, Lottoi boi Diobottootntotjooj pot Toooj, zo boo Djäooo tüi boi ntitoojjo Kiottnoit. "Voo tooo otoo otozojoo Ktjonottitoobo otobt oo Tooijtoooitt boobojo", iojt oi üpoi boizott ooob tobjoobo Uottotoioojioajjtobtottoo tüi bto Dojojoo boi Kooboo. Öo ott boo oot boo Nojoibäoboio oizoojtoo Ntioo zo boobojo oob ooob bto Nootoboi tüi bto Uottotoiooj oo Ntiooooitt zo ootzoo, oüiitoo bto Ttozojoojojoo zo otooi Ejotto zoiooooojoiobjoiioo noiboo, bto oo Tooijtoooitt nto oto Kiottnoit to Gtjonottooßitop oottittt. "Xoi tit oi, noi nti ootoi otooo ntitoojjoo Kiottnoit 4.0 noiitoboo", iojt Voijo. Öob joooo boi ntjj Toooj to otooo Jotot Uootoio ott boo Vüoobooi Tooijtoiottnoioootoioobooo Totitx ooob to btoioo Jobi oot boo Uoj pitojoo. Xotüi tonoittoit Toooj otooo znotitojjtjoo Vtjjtoooopotioj, ott boo itob bto Doijtooi bto Vobibott oo boo Goootoiobottiootoioobooo Ejoxo itoboio. Doiotti to noijoojoooo Jobi botto itob Toooj oo Totitx pototjtjt.
"""
prior = "Das Berliner Energieunternehmen Enpal und die Münchner Softwarefirma Entrix gründen ein Joint Venture / Künstliche Intelligenz soll virtuelles Kraftwerk optimieren.\n"

decipher(coded_text, prior=prior)

In [224]:
prior = """KI-Revolution im Büro erfordert Umdenken in der IT
Aktuelle IT-Infrastrukturen behindern KI-Produktivitätsgewinne. Während Unternehmen große Hoffnungen in generative KI setzen, lähmen veraltete digitale Arbeitsplätze die Entfaltung des vollen Potentials."""

coded_text = """Xto Ronojottoo boi Dipottinojt boiob Küoitjtobo Iotojjtjooz tooot toi Ntootoo - otobt otno notj bto Iooboojojto zo nootj jotitoo taooto, iooboio notj bto Vooioboo zo joojioo joiooo oob bto II-Iotioitiottoioo boi ootitoo Öotoioobooo otobt poiott tit. Zo btoioo Tijopoti tooot otoo ottoojjo jjopojo Ntobto ootoi 600 II-Eübioojitiättoo, bto otoo potiäobtjtobo Xtitiooooz zntioboo boo boboo Tinoitoojoo oo KI-joitützto Diobotttnttätiitotjoioojoo oob boi Roojttät to boo Öotoioobooo zotjt.
"""
decipher(coded_text, prior=prior)


39 dictionary variants
. |  Die 0.056865014135837555
2 dictionary variants
. Die |  Revolution 0.0015954561531543732
68 dictionary variants
. Die Revolution |  der 0.19477318227291107
1 dictionary variants
. Die Revolution der |  Arbeitswelt 0.00209758966229856
11 dictionary variants
. Die Revolution der Arbeitswelt |  durch 0.0067873080261051655
5 dictionary variants
. Die Revolution der Arbeitswelt durch |  Künstliche 5.383833255503088e-09
0 dictionary variants
Word: Intelligenz Probability: 0.9957321286201477
. Die Revolution der Arbeitswelt durch Künstliche | Intelligenz tensor(0.9957)


KeyboardInterrupt: 